<img src='Image/CBG18.png' width="220px" align="right"> <img src='Image/uff-logo.png' width="200px" align="left"> <img src='Image/logon.jpg' width="200px" align="center"> 

## <center> Synthetic data - Dike model <center> 

--------------------------------------------------------------------------------------------------------------------

In [ ]:
################ python internal packages ###############
import numpy as np
import pylab as py
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import widgets
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
#from IPython.display import Latex
#########################################################

In [ ]:
################### import external packages #################
from modules.sphere import sphere_gz
from modules.plots import rectangle, draw_prism
from modules.iplots import model_masses
##############################################################

# <center> Step # 01: Read synthetic data and the dike model info <center>

In [ ]:
# read synthetic data file using pandas package:
data = pd.read_csv('Synthetic/synthetic_data.txt', sep = ' ')
print data
# read dike file using pandas package (dike info is just for ploting issues!):
dike = pd.read_csv('Synthetic/dike_model.txt', sep = ' ')
print dike

#  <center size=3> Some useful tools of pandas: <center>


In [ ]:
data.min() # minimum values per column

In [ ]:
data.max() # maximum values per column

In [ ]:
data.sum() # sum values per column

In [ ]:
data.abs() # abs values per column

In [ ]:
data.diff() # diference values of each line

In [ ]:
data.count() # number of lines of each column

In [ ]:
data.mean() # mean of each column

In [ ]:
data.median() # median of each column

# Interpolate your data for map visualization.  This is for a non-gridded data, which is really a common situation !

In [ ]:
# set the mesh limits from real data coordinates with pandas command presented above:
xmin = data.min()[0]
print 'min x coordinate =', xmin
ymin = data.min()[1]
print 'min y coordinate =', ymin
xmax = data.max()[0]
print 'max x coordinate =', xmax
ymax = data.max()[1]
print 'max y coordinate =', ymax

# define the number of data in both x and y directions: 
nx = 100 
ny = 100 

# create a linspace for x coordinates:
x = np.linspace(xmin, xmax, nx, endpoint=True)
# and also for y coordinates:
y = np.linspace(ymin, ymax, ny, endpoint=True)
# mesh for interpolation:
X,Y = np.meshgrid(x,y)

# <center size=3>   Interpolation using scipy package : <center> 

In [ ]:
# put the read data into more mnemonic variables and convert to lists to perform calculations:
coordx = data.iloc[:,0] # x coordinate (m)
coordy = data.iloc[:,1] # y coordinate (m)
elev   = data.iloc[:,2] # elevation (m) - in this case, is one elevation for the whole data-set! pay attention!
gz     = data.iloc[:,3] # vertical attraction (mGal) 

real_coords = (coordx, coordy) # real observation positions
grid_coords = (X,Y) # interpolated observation positions

# function of scipy for interpolation:
gzi = griddata(real_coords, gz, grid_coords , method='cubic') # from now on, this will be our "real data"

# <center size=3>   Visualization of the interpolated data and the contour of the dike : <center> 

In [ ]:
# contour of the dike using pandas commands:
xmin_dike = dike.iloc[0,0]
xmax_dike = dike.iloc[0,1]
ymin_dike = dike.iloc[0,2]
ymax_dike = dike.iloc[0,3]
zmin_dike = dike.iloc[0,4]
zmax_dike = dike.iloc[0,5]
dens_dike = dike.iloc[0,6]

# create a list with the contour of the dike (IN THIS CASE, THE REAL SOURCE!)
dike2D = [xmin_dike, xmax_dike, ymin_dike, ymax_dike]

# coordinates of the dike for 3D plot:
dike3D = [xmin_dike, xmax_dike, ymin_dike, ymax_dike, zmin_dike, zmax_dike]

print 'Horizontal limits of the dike = ', dike2D
print 'Coordinates of the dike = ', dike3D

In [ ]:
# Ploting the data to see if it's ok so far:
plt.close('all')
fig = plt.figure(figsize=(12,6))
# adding axis to the figure:
ax1 = plt.axes()
fs = 14 # fontsize used in the labels

# draw the edges of the dike for better vizualization:
rectangle(ax1, dike2D, style='--k', linewidth=4, fill=None, alpha=1., label=None)

# plot the data in countor map:
plt.title('Dike model - interpolated noisy data', fontsize = fs)
p = ax1.contour(X, Y, gzi, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
ax1.clabel(p, inline=10, fontsize=10) #(only work for contour command!!!!!!)

p = ax1.contourf(X, Y, gzi, 10, cmap = plt.cm.coolwarm, vmin = np.min(gz), vmax = np.max(gz))

# Colorbar specifications:
cbar = fig.colorbar(p, aspect=40, fraction = 0.1, orientation="vertical")
cbar.set_label('Gravity Anomaly ( $ mGal$ )',fontsize=15, rotation = 90)
plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)
plt.show()

## Test #01: Place a single point mass at the very bottom of the interpretive model:

In [ ]:
# ---------- set the properties of the point mass ------------:
xc = 0.0 # meters
yc = 0.0 # meters
zc = 39000.0 # meters
rho = 2000.0 # kg/m3

# ---- save in a list to perform calculation of the gravity anomaly----:
source = ( xc, yc, zc, rho ) 
# predicted vertical attraction gz:
gz_pred = sphere_gz(X,Y,elev[0],source) 

## Plot data produced by the deep point mass and compare with the real data:

In [ ]:
# -------------------- Ploting all results obtained so far --------------------:
plt.close('all')
fig = plt.figure(figsize=(16,5))
fs = 14 # fontsize used in the labels

#-------------------- Plot the data produced by the dike model: --------------------
ax1 = plt.subplot(1, 2, 1)
plt.title('Gravity Anomaly - dike model ', fontsize = fs)

# plot isolines:
p = ax1.contour(X, Y, gzi, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

p1 = ax1.contourf(X, Y, gzi, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))
cbar1 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar1.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour jointly with the color map ---------------------:
rectangle(ax1,dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)
plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)

###########################################################################################################################
#-------------------- Plot the data produced by the deep point mass : --------------------
ax2 = plt.subplot(1, 2, 2)
plt.title('Gravity Anomaly - point mass model ', fontsize = fs)

# plot isolines:
p = ax2.contour(X, Y, gz_pred, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

# for the color map as a function of the data produced by the deep point mass:
p2 = ax2.contourf(X, Y, gz_pred, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))

#-------------------- Color bar specifications: --------------------
cbar2 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar2.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour into the contor map ---------------------:
rectangle(ax2, dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)

# -------- plot the point mass into the contor map ---------------------:
plt.scatter(xc, yc, s=rho, c='k', marker='.')

plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)

plt.show()
# save figure tests:
fig.savefig('Synthetic/test#01-deep_mass_2D.pdf', dpi=300, transparent=True, bbox_inches='tight')

#  <center> 3D Plot of model and data in slice <center>

In [ ]:
# ------- compute the difference between the "observed" data and the predicted data -------:
residuals = gzi - gz_pred

In [ ]:
plt.close('all')
%matplotlib tk
py.rcParams['contour.negative_linestyle'] = 'dashed'
fig = plt.figure(figsize=(14,10))
ax = fig.gca(projection='3d') # 3D boxplot where the figure goes

# plot the dike and store it in pbox plotting object:
pbox = draw_prism(ax, dike3D)

# --- change size projection ---
x_scale=1.
y_scale=1.
z_scale=1.
scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0
def short_proj():
    return np.dot(Axes3D.get_proj(ax), scale)
pbox.get_proj=short_proj

# ----- define labels and limits of the plotbox:
pbox.set_xlabel('Horizontal coordinate x (m)', labelpad=20 ,fontsize=fs)
pbox.set_ylabel('Horizontal coordinate y (m)', labelpad=20 ,fontsize=fs)
pbox.set_xlim( min(x), max(x) )
pbox.set_ylim( min(y), max(y) )
pbox.set_zlim(-500, 40000)
pbox.set_zlabel('Depth (m)',labelpad=20 ,fontsize=fs, rotation = 90)
pbox.view_init(35, 40)

# put z downward
pbox.invert_zaxis()

# add the deep point mass here:
pbox.scatter3D(xc, yc, zc, s=rho, c='k', marker = '.', depthshade=True)

# ------------------- vertical gravitational atraction produced by the dike ------------------:
p1 = pbox.contour(X, Y, residuals, 10, zdir='z', 
                offset=np.min(elev) , cmap='coolwarm', vmin = np.min(residuals), vmax = np.max(residuals) )
cbar1 = fig.colorbar(p1, aspect = 50, fraction = 0.01, orientation="vertical")
cbar1.set_label('Residuals ( $ mGal $ )',fontsize=fs, rotation = 90)

plt.tight_layout(True)
plt.show()
# save figure tests:
fig.savefig('Synthetic/test#01-deep_mass_3D.pdf', dpi=300, transparent=True, bbox_inches='tight')

# <center> Conclusão: <center> 
### Este é o conceito de espaço nulo, que consiste em um conjunto específico de parâmetros do modelo utilizado (massas pontuais) que não produz qualquer alteração nos dados calculados. Isso significa que alguns dos parâmetros utilizados (zc muito profundo) para o exemplo em questão não produz qualquer alteração na anomalia computada. 


## Test #02: Physical property tests - calibration of the density ranges:

### <center> Point mass with super high density: <center>

In [ ]:
# ---------- set the properties of the point mass ------------:
xc = 0.0
yc = 0.0
zc = 3000.0
rho = 20000.0
source = ( xc, yc, zc, rho ) 
# predicted vertical attraction gz:
gz_pred = sphere_gz(X,Y,elev[0],source) 
print gz_pred.shape

In [ ]:
# ------- compute gravity data for all clicked point masses:
# Calculating the gravity attraction for all point masses:
gz = 0.0
# range of all masses:
source = [ xc, yc, zc, rho ] 
#print source
gz_pred = sphere_gz(X, Y, elev[0], source) # sum the effect of all masses 

## <Center> Plot 2D contour map to check for the data-misfit. <center>

In [ ]:
# -------------------- Ploting all results obtained so far --------------------:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(16,5))
fs = 14 # fontsize used in the labels

#-------------------- Plot the data produced by the dike model: --------------------
ax1 = plt.subplot(1, 2, 1)
plt.title('Gravity Anomaly - dike model ', fontsize = fs)

# plot isolines:
p = ax1.contour(X, Y, gzi, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

p1 = ax1.contourf(X, Y, gzi, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))
cbar1 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar1.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour jointly with the color map ---------------------:
rectangle(ax1,dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)
plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)

###########################################################################################################################
#-------------------- Plot the data produced by the deep point mass : --------------------
ax2 = plt.subplot(1, 2, 2)
plt.title('Gravity Anomaly - point mass model ', fontsize = fs)

# plot isolines:
p = ax2.contour(X, Y, gz_pred, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

# for the color map as a function of the data produced by the deep point mass:
p2 = ax2.contourf(X, Y, gz_pred, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))

#-------------------- Color bar specifications: --------------------
cbar2 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar2.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour into the contor map ---------------------:
rectangle(ax2, dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)

# -------- plot the point mass into the contor map ---------------------:
plt.scatter(xc, yc, s=1000.0, c='k', marker='.')

plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)



plt.show()
# save figure tests:
fig.savefig('Synthetic/test#02-High_Density_Mass_2D.pdf', dpi=300, transparent=True, bbox_inches='tight')

#  <center> 3D Plot of model and data in slice <center>

In [ ]:
# ------- compute the difference between the "observed" data and the predicted data -------:
residuals = gzi-gz_pred

In [ ]:
%matplotlib tk
py.rcParams['contour.negative_linestyle'] = 'dashed'
plt.close('all')
fig = plt.figure(figsize=(14,10))
ax = fig.gca(projection='3d') # 3D boxplot where the figure goes

# plot the dike and store it in pbox plotting object:
pbox = draw_prism(ax, dike3D)

# --- change size projection ---
x_scale=1.
y_scale=1.
z_scale=1.
scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0
def short_proj():
    return np.dot(Axes3D.get_proj(ax), scale)
pbox.get_proj=short_proj

# ----- define labels and limits of the plotbox:
pbox.set_xlabel('Horizontal coordinate x (m)', labelpad=20 ,fontsize=fs)
pbox.set_ylabel('Horizontal coordinate y (m)', labelpad=20 ,fontsize=fs)
pbox.set_xlim( min(x), max(x) )
pbox.set_ylim( min(y), max(y) )
pbox.set_zlim(-500,10000)
pbox.set_zlabel('Depth (m)',labelpad=20 ,fontsize=fs, rotation = 90)
pbox.view_init(15, 55)

# put z downward
pbox.invert_zaxis()

# ------------------- vertical gravitational atraction produced by the dike ------------------:
p1 = pbox.contour(X, Y, residuals, 10, zdir='z', 
                offset=np.min(elev), cmap='coolwarm', vmin = np.min(residuals), vmax = np.max(residuals) )
cbar1 = fig.colorbar(p1, aspect = 50, fraction = 0.01, orientation="vertical")
cbar1.set_label('Residuals ( $ mGal $ )',fontsize=16, rotation = 90)

###########################################################################################################################
# ------------------- vertical gravitational atraction produced by the point mass--------:
#p2 = pbox.contour(X, Y, gz_pred, 10, zdir='z', offset=np.min(elev) , colors='k', linewidths = 0.5, 
#                  vmin=np.min(gzi), vmax=np.max(gzi) )

# add the deep point mass here:
pbox.scatter3D(xc, yc, zc, s=rho , c='k', marker = '.')

# ------------- Color bar specifications ---------------------------:
#cbar2 = fig.colorbar(p1, aspect = 50, fraction = 0.01, orientation="vertical")
#cbar2.set_label('Gravity Anomaly ( $ mGal $ )',fontsize=fs, rotation = 90)
#cbar2.add_lines(p2)

#plt.clabel(p2, inline=1, fontsize=8)
#plt.tight_layout(True)
plt.show()
# save figure tests:
fig.savefig('Synthetic/test#02-High_Density_Mass_3D.pdf', dpi=300, transparent=True, bbox_inches='tight')

### <center> Test #03 :  Point mass with super low density: <center>

In [ ]:
# ---------- set the properties of the point mass ------------:
xc = 0.0
yc = 0.0
zc = 3000.0
rho = 500.0
source = ( xc, yc, zc, rho ) 
# predicted vertical attraction gz:
gz_pred = sphere_gz(X,Y,elev[0],source) 
print gz_pred.shape

In [ ]:
# ------- compute gravity data for all clicked point masses:
# Calculating the gravity attraction for all point masses:
gz = 0.0
# range of all masses:
source = [ xc, yc, zc, rho ] 
#print source
gz_pred = sphere_gz(X, Y, elev[0], source) # sum the effect of all masses 

## <Center> Plot 2D contour map to check for the data-misfit. <center>

In [ ]:
# -------------------- Ploting all results obtained so far --------------------:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(16,5))
fs = 14 # fontsize used in the labels

#-------------------- Plot the data produced by the dike model: --------------------
ax1 = plt.subplot(1, 2, 1)
plt.title('Gravity Anomaly - dike model ', fontsize = fs)

# plot isolines:
p = ax1.contour(X, Y, gzi, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

p1 = ax1.contourf(X, Y, gzi, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))
cbar1 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar1.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour jointly with the color map ---------------------:
rectangle(ax1,dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)
plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)

###########################################################################################################################
#-------------------- Plot the data produced by the deep point mass : --------------------
ax2 = plt.subplot(1, 2, 2)
plt.title('Gravity Anomaly - point mass model ', fontsize = fs)

# plot isolines:
p = ax2.contour(X, Y, gz_pred, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

# for the color map as a function of the data produced by the deep point mass:
p2 = ax2.contourf(X, Y, gz_pred, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))

#-------------------- Color bar specifications: --------------------
cbar2 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar2.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour into the contor map ---------------------:
rectangle(ax2, dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)

# -------- plot the point mass into the contor map ---------------------:
plt.scatter(xc, yc, s=rho, c='k', marker='.')

plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)
plt.show()
# save figure tests:
fig.savefig('Synthetic/test#03-Low_Density_Mass_2D.pdf', dpi=300, transparent=True, bbox_inches='tight')

#  <center> 3D Plot of model and data-misfit in slice <center>

In [ ]:
# ------- compute the difference between the "observed" data and the predicted data -------:
residuals = gzi - gz_pred

In [ ]:
%matplotlib tk
py.rcParams['contour.negative_linestyle'] = 'dashed'
plt.close('all')
fig = plt.figure(figsize=(14,10))
ax = fig.gca(projection='3d') # 3D boxplot where the figure goes

# plot the dike and store it in pbox plotting object:
pbox = draw_prism(ax, dike3D)

# --- change size projection ---
x_scale=1.
y_scale=1.
z_scale=1.
scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0
def short_proj():
    return np.dot(Axes3D.get_proj(ax), scale)
pbox.get_proj=short_proj

# ----- define labels and limits of the plotbox:
pbox.set_xlabel('Horizontal coordinate x (m)', labelpad=20 ,fontsize=fs)
pbox.set_ylabel('Horizontal coordinate y (m)', labelpad=20 ,fontsize=fs)
pbox.set_xlim( min(x), max(x) )
pbox.set_ylim( min(y), max(y) )
pbox.set_zlim(-500,10000)
pbox.set_zlabel('Depth (m)',labelpad=20 ,fontsize=fs, rotation = 90)
pbox.view_init(15, 55)

# put z downward
pbox.invert_zaxis()

# ------------------- vertical gravitational atraction produced by the dike ------------------:
p1 = pbox.contour(X, Y, residuals, 10, zdir='z', 
                offset=np.min(elev), cmap='coolwarm', vmin = np.min(residuals), vmax = np.max(residuals) )
cbar1 = fig.colorbar(p1, aspect = 50, fraction = 0.01, orientation="vertical")
cbar1.set_label('Residuals ( $ mGal $ )',fontsize=16, rotation = 90)

###########################################################################################################################
# ------------------- vertical gravitational atraction produced by the point mass--------:
#p2 = pbox.contour(X, Y, gz_pred, 10, zdir='z', offset=np.min(elev) , colors='k', linewidths = 0.5, 
#                  vmin=np.min(gzi), vmax=np.max(gzi) )

# add the deep point mass here:
pbox.scatter3D(xc, yc, zc, s=rho , c='k', marker = '.')

# ------------- Color bar specifications ---------------------------:
#cbar2 = fig.colorbar(p1, aspect = 50, fraction = 0.01, orientation="vertical")
#cbar2.set_label('Gravity Anomaly ( $ mGal $ )',fontsize=fs, rotation = 90)
#cbar2.add_lines(p2)

plt.clabel(p2, inline=1, fontsize=8)
plt.tight_layout(True)
plt.show()
# save figure tests:
fig.savefig('Synthetic/test#03-Low_Density_Mass_3D.pdf', dpi=300, transparent=True, bbox_inches='tight')

## Test #04: Using the click method for modeling with many point masses 

In [ ]:
#--------- define the plotting area for both (x,y) and (depth, density)---------:
area_xy = [min(x) , max(x), min(y), max(y)]

# based on the knowledge about the rock densities and common depth investigations!
area_rhoz = [2800.0, 3000.0, np.min(elev), 7000.0]

# dike info for plotting issues:
true_source = [xmin_dike, xmax_dike, ymin_dike, ymax_dike, zmin_dike, zmax_dike, dens_dike]

print 'area_xy=', area_xy
print 'area_rhoz=', area_rhoz
print 'true source=', true_source

In [ ]:
# call function for picking x,y,z,rho values from the plot areas:
%matplotlib tk
xc,yc,zc,rho = model_masses(area_xy, area_rhoz, newclicks=None, dike=true_source)

In [ ]:
print 'These numbers should be the same =', len(xc), len(yc), len(zc), len(rho)
nmasses = len(xc)
print 'number of clicks performed =', nmasses


In [ ]:
# ------- compute gravity data for all clicked point masses:
# Calculating the gravity attraction for all point masses:
gz_pred = 0.0
# range of all masses:
for i in range( len(xc) ):
    source=[]
    source = [ xc[i], yc[i], zc[i], rho[i]/nmasses ]  # calling one source a time:
    print source
    gz_pred += sphere_gz(X, Y, data.mean()['elev(m)'], source) # sum the effect of all masses 

## <Center> Plot 2D contour map to check for the data-fit. <center>

In [ ]:
# -------------------- Ploting all results obtained so far --------------------:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(16,5))
fs = 14 # fontsize used in the labels

#-------------------- Plot the data produced by the dike model: --------------------
ax1 = plt.subplot(1, 2, 1)
plt.title('Gravity Anomaly - dike model ', fontsize = fs)

# plot isolines:
p = ax1.contour(X, Y, gzi, 10, colors='k', linewidths = 0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

p1 = ax1.contourf(X, Y, gzi, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))
cbar1 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar1.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour jointly with the color map ---------------------:
rectangle(ax1,dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)
plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)

###########################################################################################################################
#-------------------- Plot the data produced by the deep point mass : --------------------
ax2 = plt.subplot(1, 2, 2)
plt.title('Gravity Anomaly - point mass model ', fontsize = fs)

# plot isolines:
p = ax2.contour(X, Y, gz_pred, 10, colors='k', linewidths=0.2, vmin=np.min(gzi), vmax=np.max(gzi) )

# display the lines and the respective value of each line:
plt.clabel(p, inline=1, fontsize=10) #(only work for contour command!!!!!!)

# for the color map as a function of the data produced by the deep point mass:
p2 = ax2.contourf(X, Y, gz_pred, 10, cmap = plt.cm.coolwarm, vmin = np.min(gzi), vmax = np.max(gzi))

#-------------------- Color bar specifications: --------------------
cbar2 = plt.colorbar(p1, aspect=30, fraction = 0.05, orientation="horizontal")
cbar2.set_label('$g_z $ ( $ mGal$ )',fontsize=15, rotation = 0)

# -------- plot the dike contour into the contor map ---------------------:
rectangle(ax2, dike2D, style='--k', linewidth=4, fill= None, alpha=1., label=None)

# -------- plot the point mass into the contor map ---------------------:
plt.scatter(xc, yc, s=rho, c='k', marker='.')

plt.ylabel('Horizontal coordinate Y (m)', fontsize = fs)
plt.xlabel('Horizontal coordinate X (m)', fontsize = fs)

plt.show()
# save figure tests:
fig.savefig('Synthetic/test#04-multiple_masses_2D.pdf', dpi=300, transparent=True, bbox_inches='tight')

#  <center> 3D Plot of model and data-misfit in slice <center>

In [ ]:
# ------- compute the difference between the "observed" data and the predicted data -------:
residuals = gzi - gz_pred

In [ ]:
plt.close('all')
%matplotlib tk
py.rcParams['contour.negative_linestyle'] = 'dashed'

# define figure instance:
fig = plt.figure(figsize=(14,10))

# 3D boxplot where the figure goes
ax = fig.gca(projection='3d') 

# plot the dike and store it in pbox plotting object:
pbox = draw_prism(ax, dike3D)

# --- change size projection ---
x_scale=1.
y_scale=1.
z_scale=1.
scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0
def short_proj():
    return np.dot(Axes3D.get_proj(ax), scale)
pbox.get_proj=short_proj

# ----- define labels and limits of the plotbox:
pbox.set_xlabel('Horizontal coordinate x (m)', labelpad=20 ,fontsize=fs)
pbox.set_ylabel('Horizontal coordinate y (m)', labelpad=20 ,fontsize=fs)
pbox.set_xlim( min(x), max(x) )
pbox.set_ylim( min(y), max(y) )
pbox.set_zlim(-500,10000)
pbox.set_zlabel('Depth (m)',labelpad=20 ,fontsize=fs, rotation = 90)
pbox.view_init(30, 55)

# put z downward
pbox.invert_zaxis()

# ------------------- vertical gravitational atraction produced by the dike ------------------:
p1 = pbox.contour(X, Y, residuals, 10, zdir='z', 
                offset=np.min(elev), cmap='coolwarm', vmin = np.min(residuals), vmax = np.max(residuals) )
cbar1 = fig.colorbar(p1, aspect = 50, fraction = 0.01, orientation="vertical")
cbar1.set_label('Residuals ( $ mGal $ )',fontsize=16, rotation = 90)

# add the deep point mass here:
pbox.scatter3D(xc, yc, zc, s=rho , c='k', marker = '.')

plt.tight_layout(True)
plt.show()
# save figure tests:
fig.savefig('Synthetic/test#04-multiple_masses_3D.pdf', dpi=300, transparent=True, bbox_inches='tight')